In [1]:
%pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu11==24.2.* dask-cudf-cu11==24.2.* cuml-cu11==24.2.* \
    cugraph-cu11==24.2.* cuspatial-cu11==24.2.* cuproj-cu11==24.2.* \
    cuxfilter-cu11==24.2.* cucim-cu11==24.2.* pylibraft-cu11==24.2.* \
    raft-dask-cu11==24.2.*



Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.9/460.9 MB 16.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 6.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 5.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 MB 36.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 96.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 MB 38.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB

In [10]:
import os
import re
from datetime import datetime

PROVIDERS = {
    "google": {
        "raw": "Takeout/My Activity/Search",
        "parsed": "google/search_history",
        "summary": "google/search_history_summary",
        "context": ""
    }
}


def get_filenames(
    kind="parsed", start_date=None, end_date=None, provider="google"
):
    directory = os.path.join(kind, PROVIDERS[provider][kind])
    if start_date is not None:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if end_date is not None:
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    file_pattern = r"^(\d{4}-\d{2}-\d{2})\.csv$"

    def is_date_in_range(file_date):
        if start_date is None and end_date is None:
            return True
        else:
            return start_date <= datetime.strptime(file_date, "%Y-%m-%d") <= end_date

    filenames = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            match = re.match(file_pattern, file)
            if match:
                file_date = match.groups()[0]
                if is_date_in_range(file_date):
                    filenames.append(os.path.join(root, file))

    return filenames

In [ ]:
# load all parsed data into a single dataframe
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=["date", "title", "embedding"])

for filename in get_filenames():
    date = filename.split("/")[-1].split(".")[0]

    try:
        embeddings = np.load(f"embeddings/{date}.npy")

        tmp_df = pd.read_csv(filename)
        tmp_df = tmp_df.drop_duplicates(subset="title")

        tmp_df = tmp_df[~tmp_df["title"].str.contains("Visited ")]
        tmp_df = tmp_df[~tmp_df["title"].str.contains("Used ")]
        tmp_df = tmp_df[~tmp_df["title"].str.contains("Defined ")]

        tmp_df["title"] = tmp_df["title"].str.replace("Searched for ", "")

        tmp_df["date"] = date  
        tmp_df["embedding"] = embeddings.tolist()

        df = pd.concat([df, tmp_df])
    except Exception as e:
        print(e)
        pass

In [ ]:
%pip install hdbscan

In [21]:
import numpy as np
import cudf
import cuml
import cupy as cp  # Import cuPy

df_gpu = cudf.DataFrame.from_pandas(df)

embeddings_np = np.array(df["embedding"].to_list())

# Then, convert the NumPy array to a cuPy array
embeddings_gpu = cp.asarray(embeddings_np)
# UMAP for dimensionality reduction
umap_model = cuml.UMAP(n_neighbors=15,
                       n_components=100, 
                       min_dist=0.1, 
                       metric='cosine')
reduced_data_gpu = umap_model.fit_transform(embeddings_gpu)

# HDBSCAN for clustering
clusterer = cuml.cluster.HDBSCAN(min_cluster_size=5, # minimum size of clusters
                                 gen_min_span_tree=True) # useful for visualization, if supported
cluster_labels_gpu = clusterer.fit_predict(reduced_data_gpu)

In [ ]:
df_gpu["reduced_data"] = reduced_data_gpu.get().tolist()
df_gpu["cluster_labels"] = cluster_labels_gpu.get().tolist()
df_gpu.index = range(len(df_gpu))
df_gpu.to_json("clustered.json")

In [ ]:
import pandas as pd
df = pd.read_json("../_data/clustered.json")

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)
filtered_df = df[df["cluster_labels"] != -1]
filtered_df = filtered_df[["date", "title", "cluster_labels"]]
filtered_df = filtered_df.sort_values(by=["cluster_labels", "date"])
filtered_df